### Scenario - A retail company receives daily updates for its product catalog, including new products, price changes, and discontinued items. Instead of overwriting the entire catalog or simply appending new records, they need to upsert the incoming data by updating existing products with the latest information and inserting new ones, ensuring the catalog remains accurate and up to date in real time.

### **Quering Source**

In [0]:
%sql
SELECT * FROM pyspark_cata.source.products

id,name,price,category,updatedDate
1,iPhone,1000,electronics,2025-08-19T06:30:41.376Z
2,Macbook,2000,electronics,2025-08-19T06:30:41.376Z
3,T-Shirt,50,clothing,2025-08-19T06:30:41.376Z
4,Shirt,100,clothing,2025-08-19T06:30:41.376Z
5,Pants,150,clothing,2025-08-19T06:30:41.376Z
5,Trousers,150,clothing,2025-08-23T11:12:07.314Z


In [0]:
# df = spark.read.table('pyspark_cata.source.products')
df = spark.sql("select * from pyspark_cata.source.products")
df.display()

id,name,price,category,updatedDate
1,iPhone,1000,electronics,2025-08-19T06:30:41.376Z
2,Macbook,2000,electronics,2025-08-19T06:30:41.376Z
3,T-Shirt,50,clothing,2025-08-19T06:30:41.376Z
4,Shirt,100,clothing,2025-08-19T06:30:41.376Z
5,Pants,150,clothing,2025-08-19T06:30:41.376Z
5,Trousers,150,clothing,2025-08-23T11:12:07.314Z


In [0]:
from pyspark.sql.window import Window
from pyspark.sql.functions import *

In [0]:
#Dedup
df = df.withColumn('dedup',row_number().over(
    Window.partitionBy('id')\
    .orderBy(desc('updatedDate'))
    ))
df.display()

id,name,price,category,updatedDate,dedup
1,iPhone,1000,electronics,2025-08-19T06:30:41.376Z,1
2,Macbook,2000,electronics,2025-08-19T06:30:41.376Z,1
3,T-Shirt,50,clothing,2025-08-19T06:30:41.376Z,1
4,Shirt,100,clothing,2025-08-19T06:30:41.376Z,1
5,Trousers,150,clothing,2025-08-23T11:12:07.314Z,1
5,Pants,150,clothing,2025-08-19T06:30:41.376Z,2


In [0]:
#Filter and keep the latest entry (since orderBy desc, latest will be 1) and drop dedup column
df = df.filter(
    col('dedup')==1
).drop('dedup')
df.display()

id,name,price,category,updatedDate
1,iPhone,1000,electronics,2025-08-19T06:30:41.376Z
2,Macbook,2000,electronics,2025-08-19T06:30:41.376Z
3,T-Shirt,50,clothing,2025-08-19T06:30:41.376Z
4,Shirt,100,clothing,2025-08-19T06:30:41.376Z
5,Trousers,150,clothing,2025-08-23T11:12:07.314Z


### **Upserts**

In [0]:
#creating delta objects
from delta.tables import DeltaTable

try:
    dlt_obj = DeltaTable.forPath(spark,'/Volumes/pyspark_cata/source/db_volume/products_sink/')

    dlt_obj.alias('trg').merge(
        df.alias('src'),
        'src.id = trg.id'
    ).whenMatchedUpdateAll(condition='src.updatedDate >= trg.updatedDate')\
        .whenNotMatchedInsertAll()\
        .execute()
    print('Upserting')
except:
    df.write.format('delta')\
        .mode('Overwrite')\
        .save('/Volumes/pyspark_cata/source/db_volume/products_sink/')
    print('Writing')

Upserting


In [0]:
%sql
SELECT * FROM delta.`/Volumes/pyspark_cata/source/db_volume/products_sink`

id,name,price,category,updatedDate
1,iPhone,1000,electronics,2025-08-19T06:30:41.376Z
4,Shirt,100,clothing,2025-08-19T06:30:41.376Z
3,T-Shirt,50,clothing,2025-08-19T06:30:41.376Z
2,Macbook,2000,electronics,2025-08-19T06:30:41.376Z
5,Trousers,150,clothing,2025-08-23T11:12:07.314Z
